In [15]:
from __init__ import log, config, path
from utils.sqlite import SQLite
import pandas as pd
from datetime import datetime, timedelta

path_db = r"D:\DevRPA\Clientes\Soufer\Projetos\RPA_Soufer_Retorno_Extrato_v2\data\db_rpa_soufer.db"
db_name = 'rpa_soufer_queue_retorno_extrato'

db = SQLite(path_db=path_db,
            db_name=db_name)

2024-11-21 23:41:20    [INFO] - Conexão com o banco de dados estabelecida com sucesso.


In [9]:
db.close_db_connection()

2024-11-21 23:39:53    [INFO] - Conexão com o banco de dados fechada com sucesso.


In [16]:
db.create_table()

2024-11-21 23:41:22    [INFO] - Tabela 'rpa_soufer_queue_retorno_extrato' criada com sucesso.


In [17]:
from datetime import datetime, timedelta
from utils import Functions

def insert_database_queue():

    df = pd.read_excel(path.database('Relatorio_Processamento_Base.xlsx'))
    
    data_atual = datetime.today()
    

    # Verificar se é segunda-feira
    if data_atual.weekday() == 0:
        # Subtrair dois dias da data atual para processar com a data de sábado
        data_editada = Functions.data_formatada(time_delta=2)
    else:
        # Manter a data do dia atual
        data_editada = data_atual
        
    # Define os padrões de substituição
    padrao_DDMMAA = 'DDMMAA'
    padrao_AAMMDD = 'AAMMDD'
    padrao_DDMM = 'DDMM'
    padrao_DDMMAAAA = 'DDMMAAAA'
    
    data_editada_DDMMAA = data_editada.strftime('%d%m%y')
    data_editada_AAMMDD = data_editada.strftime('%y%m%d')
    data_editada_DDMM = data_editada.strftime('%d%m')
    data_editada_DDMMAAAA = data_editada.strftime('%d%m%Y')

    for index, row in df.iterrows():
        
        nome_arquivo = str(row['Arquivo']).replace(padrao_DDMMAAAA, data_editada_DDMMAAAA)
        nome_arquivo = nome_arquivo.replace(padrao_DDMMAA, data_editada_DDMMAA)
        nome_arquivo = nome_arquivo.replace(padrao_AAMMDD, data_editada_AAMMDD)
        nome_arquivo = nome_arquivo.replace(padrao_DDMM, data_editada_DDMM)
        
        if nome_arquivo == f'EXT_745_17155789_{data_editada_AAMMDD}_00000.RET':
            nome_arquivo = f'EXT_745_17155789_{data_atual.strftime('%y%m%d')}_00000.RET'
        
        data = {
            'data_execucao': datetime.now().strftime('%d/%m/%Y'),
            'data_arquivo': data_editada.strftime('%d/%m/%Y'),
            'banco': row['Banco'],
            'empresa': row['Empresa'],
            'formato_extrato': row['Formato do extrato'],
            'caminho_arquivo': row['Caminho do Arquivo'],
            'arquivo': nome_arquivo,
            'status': 'PENDENTE'
        }

        db.insert(data)
        
insert_database_queue()

In [18]:
db.drop_table()

2024-11-21 23:43:00    [INFO] - Todos os dados foram removidos da fila com sucesso.
